The reflection method.
We know that Peano nats are kind of the same thing as Ints
Peano nats are more syntactic.

It would be a nice convenience feature for 



In [6]:
from z3 import *

def lemma(thm, by=[], admit=False):
    if not admit:
        prove(Implies(And(by), thm))
    else:
        print("Admitted!!", thm)
    return thm
B = BoolSort()
Z = IntSort()

In [7]:

Nat = Datatype("Nat")
Nat.declare('zero')
Nat.declare('succ', ('pred', Nat))
Nat = Nat.create()

n,m,k  = Consts('n m k', Nat)
reify = Function('reify', Nat, Z)
reify_def = ForAll([n], reify(n) == If(Nat.is_zero(n), 0, reify(Nat.pred(n)) + 1))

x,y = Ints("x y")
reflect = Function('reflect', Z, Nat)
reflect_def = ForAll([x], reflect(x) == If(x <= 0, Nat.zero, Nat.succ(reflect(x-1))))

#def recurse(name,base,step):
#    f = Function(name, Nat, IntSort())
#    n = FreshInt()
#    return f, ForAll([n], If(Nat.is_zero(n), base, step(n, f(Nat.pred(n),base,step))))

def induct(P):
    return Implies(And(P(Nat.zero), ForAll([n], Implies(P(n), P(Nat.succ(n))))),
                   #-----------------------------------------------------------
                   ForAll([n], P(n)))

reflect_reify = lemma(ForAll([n], reflect(reify(n)) == n), [reflect_def, reify_def, induct(lambda n: reflect(reify(n)) == n)])

reify_ge_0 = lemma(ForAll([n], reify(n) >= 0), [reify_def, induct(lambda n: reify(n) >= 0)])


zero_homo = lemma(reify(Nat.zero) == 0, [reify_def])
succ_homo = lemma(ForAll([n], reify(Nat.succ(n)) == 1 + reify(n)), [reify_def, induct(lambda n: reify(Nat.succ(n)) == 1 + reify(n))])

plus = Function('plus', Nat, Nat, Nat)
plus_def = ForAll([n, m], plus(n, m) == reflect(reify(n) + reify(m)))



plus_0_x = lemma(ForAll([n], plus(Nat.zero, n) == n), [reflect_reify, zero_homo, plus_def]) 
plus_succ_x = lemma(ForAll([n,m], plus(Nat.succ(n), m) == Nat.succ(plus(n,m))), [reflect_reify, reflect_def, reify_def, succ_homo, plus_def])


plus_homo_case_succ = lemma(ForAll([n,m], Implies(reify(plus(n,m)) == reify(n) + reify(m) ,
                                                  reify(plus(Nat.succ(n),m)) == reify(Nat.succ(n)) + reify(m) )), [reify_def, plus_succ_x])
#plus_homo = lemma(ForAll([n,m], reify(reflect(reify(n) + reify(m))) == reify(n) + reify(m)), [reflect_reify, reflect_def, reify_def, induct(lambda n: ForAll([m], reify(reflect(reify(n) + reify(m))) == reify(n) + reify(m)))])
plus_homo = lemma(ForAll([n,m], reify(plus(n,m)) == reify(n) + reify(m)), 
                  [plus_def, reflect_reify, reflect_def, reify_def, plus_homo_case_succ, induct(lambda n: ForAll([m], reify(plus(n,m)) == reify(n) + reify(m)))], admit=False)
#plus_l_expand = lemma(ForAll([n, m, k], plus(plus(n, m), k) == reflect(reify(reflect(reify(n) + reify(m))) + reify(k))), [plus_def])



#_2 = lemma(ForAll([n,m,k], reflect(reify(reflect(reify(n) + reify(m))) + reify(k)) == reflect(reify(n) + reify(m) + reify(k))), [reflect_reify, plus_homo])
plus_comm = lemma(ForAll([n, m], plus(n, m) == plus(m, n)), [plus_def])
plus_assoc = lemma(ForAll([n, m, k], plus(plus(n, m), k) == plus(n, plus(m, k))), [plus_def, plus_homo])


one = Nat.succ(Nat.zero)
two = Nat.succ(Nat.succ(Nat.zero))


proved
proved
proved
proved
proved
proved
proved
proved
proved
proved


In [8]:
mul = Function("mul", Nat,Nat, Nat)
mul_def = ForAll([n,m], mul(n,m) == reflect(reify(n) * reify(m)))
                 
mul_0_x = lemma(ForAll([m], mul(Nat.zero,m) == Nat.zero), [reflect_def, reify_def, mul_def])

#mul_succ_x = lemma(ForAll([n,m], mul(Nat.succ(n), m) == reflect(reify(Nat.succ(n))*reify(m))), [plus_homo, mul_def, plus_def, succ_homo, reflect_reify, reflect_def, reify_def])
mul_succ_x_lemma = lemma(ForAll([n,m], reflect(reify(Nat.succ(n)) * reify(m)) == reflect((1 + reify(n)) * reify(m))), [succ_homo])
mul_succ_x_lemma2 = lemma(ForAll([n,m], reflect((1 + reify(n)) * reify(m)) == reflect(reify(m) + reify(n) * reify(m))))

# Hmmm. Need to think about how to dignify this principal.
reify_reflect = lemma(ForAll([x], Implies(x >= 0, reify(reflect(x)) == x)), [reify_def, reflect_def], admit=True)

mul_succ_x_lemma3 = lemma(ForAll([n,m], reflect(reify(m) + reify(n) * reify(m)) == reflect(reify(m) + reify(reflect(reify(n) * reify(m))))), 
                          [reflect_reify, reify_reflect, reflect_def, reify_def])

mul_succ_x_lemma4 = lemma(ForAll([n,m], reflect(reify(m) + reify(reflect(reify(n) * reify(m)))) == plus(m, mul(n,m))), 
                          [plus_def, mul_def])
#mul_succ_x = lemma(ForAll([n,m], mul(Nat.succ(n), m) == plus(m, mul(n,m))), [mul_def, reify_reflect, mul_succ_x_lemma, mul_succ_x_lemma2, mul_succ_x_lemma3, mul_succ_x_lemma2, plus_homo, reflect_reify, plus_def])
mul_succ_x = lemma(ForAll([n,m], mul(Nat.succ(n), m) == plus(m, mul(n,m))), [mul_def, plus_def, mul_succ_x_lemma, mul_succ_x_lemma2, mul_succ_x_lemma3, mul_succ_x_lemma4])

proved
proved
proved
Admitted!! ForAll(x, Implies(x >= 0, reify(reflect(x)) == x))
proved
proved
proved


Could cvc5 show some of the things that z3 couldn't in my post?
Can I get the even odd style proof to go through

https://coq.inria.fr/doc/v8.10/stdlib/Coq.Init.Nat.html

In [18]:
lt = Function("lt", Nat, Nat, B)
lt_def = ForAll([n,m], lt(n,m) == reify(n) < reify(m))

max_ = Function("max", Nat, Nat, Nat)
max_def = ForAll([n,m], max_(n,m) == If(lt(n,m), m, n))

even = Function("even", Nat, B)
even_def = ForAll([n], even(n) == If(Nat.is_zero(n), True, 
                                  If(Nat.is_zero(Nat.pred(n)), False, 
                                     even(Nat.pred(Nat.pred(n))))))

odd = Function("odd", Nat, B)
odd_def = ForAll([n], odd(n) == If(Nat.is_zero(n), False, 
                                  If(Nat.is_zero(Nat.pred(n)), True, 
                                     odd(Nat.pred(Nat.pred(n))))))
 
even_zero = lemma(even(Nat.zero), [even_def])
not_even_one = lemma(Not(even(one)), [even_def])
odd_even = lemma(odd(one), [odd_def])
two_even = lemma(even(two), [even_def])

even_or_odd = lemma(ForAll([n], even(n) != odd(n)), [even_def, odd_def, induct(lambda n: even(n) != odd(n))])


proved
proved
proved
proved
failed to prove


Z3Exception: model is not available

In [21]:
def calc(vars, *args, by=[]):
    lemmas = []
    for lhs,rhs in zip(args[:-1],args[1:]) :
        lemmas.append(lemma(ForAll(vars, lhs == rhs), by))
    return lemma(ForAll(vars, args[0] == args[-1]), by=lemmas)

#calc([n,m,k], one + (a + b), m, k)


counterexample
[]
counterexample
[]
proved


ForAll([n, m, k], n == k)

In [39]:
type(n).__mro__

AstRef.childs = property(lambda self: self.children())
AstRef.head = property(lambda self: self.decl())
match plus(n,n):
    case AstRef(head=mul, childs=foo):
        print(foo)
        print(x)
        print(mul)
mul

[n, n]
plus
plus


plus

In [43]:
Point = Datatype("Point")
Point.declare("mk-point", ("x", Z), ("y", Z))
Point = Point.create()

DSegment = Datatype("DSegment")
DSegment.declare("mk-dseg", ("a", Point), ("b", Point))
DSegment = DSegment.create()

# line is equivalence class of segments.
# we can't really give canonical coordinates to lines.
# ax + by = c is scale invariant
# y = mx + b can't represent vertical lines.
# y = lam (a - b) + b  could use other points

 





Maybe more intriguing is to do projective geomtry first though


In [46]:
Point = DeclareSort("Point")
R = RealSort()
x,y,z,p,q,w = Consts("x y z p q w", R)
mkpoint = Function("mk-point", R, R, R, Point)
point_equal = ForAll([x,y,z,p,q,w], And(x*q - y*p == 0, y*w - z*q == 0, z*p - x*w == 0) == (mkpoint(x,y,z) == mkpoint(p,q,w)))
X = Function("X", Point, R)
Y = Function("Y", Point, R)
Z = Function("Z", Point, R)
mkpoint_XYZ = ForAll([p], mkpoint(X(p), Y(p), Z(p)) == p)

# right to left is trivial.
# left to right
coord_unique = ForAll([p],And(X(p) == X(q), Y(p) == Y(q), Z(p) == Z(p))  == (p == q))

# also could do exists lam, v = lam * w as equality.